1. 用鳶尾花全部的欄位來做羅吉斯迴歸模型預測。
(1) 請將資料做標準化，並輸出正確率、混亂矩陣和綜合報告。請注意，全部
資料裡一共有三個類別的花朵。
(2) 請預測這筆資料的結果是什麼？ [2,1,3,3]。（提示：請注意資料維度問
題。）
(3) 請預測這兩筆資料的結果是什麼？ [2,1,3,3],[4,5,7,8]。（提示：請注意資料
維度問題。）

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
# 載入鳶尾花數據集
iris = load_iris()
X = iris.data
y = iris.target

# 將資料標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 分割訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

# 訓練羅吉斯迴歸模型
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

# 預測測試集
y_pred = model.predict(X_test)

print("正確率:", accuracy_score(y_test, y_pred))
print("混淆矩陣:\n",pd.DataFrame(confusion_matrix(y_test, y_pred),
      index=['實際1', '實際2','實際3'], columns=['預測1', '預測2','預測3']))
print("綜合報告:\n", classification_report(y_test, y_pred))

#(2) 請預測這筆資料的結果是什麼？ [2,1,3,3]。（提示：請注意資料維度問 題。）
import numpy as np
# 轉換樣本資料成為二維陣列
sample_data = np.array([[2, 1, 3, 3]])
sample_data_scaled = scaler.transform(sample_data)

# 預測樣本資料
prediction = model.predict(sample_data_scaled)
print("資料 [2,1,3,3] 的預測結果:", iris.target_names[prediction])

#(3) 請預測這兩筆資料的結果是什麼？ [2,1,3,3],[4,5,7,8]。（提示：請注意資料 維度問題。）
# 轉換資料成為二維陣列
data_to_predict = np.array([[2, 1, 3, 3], [4, 5, 7, 8]])
data_to_predict_scaled = scaler.transform(data_to_predict)

# 預測資料
predictions = model.predict(data_to_predict_scaled)
print("資料 [2,1,3,3] 和 [4,5,7,8] 的預測結果:", [iris.target_names[pred] for pred in predictions])

正確率: 0.88
混淆矩陣:
      預測1  預測2  預測3
實際1   19    0    0
實際2    0   10    5
實際3    0    1   15
綜合報告:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.91      0.67      0.77        15
           2       0.75      0.94      0.83        16

    accuracy                           0.88        50
   macro avg       0.89      0.87      0.87        50
weighted avg       0.89      0.88      0.88        50

資料 [2,1,3,3] 的預測結果: ['versicolor']
資料 [2,1,3,3] 和 [4,5,7,8] 的預測結果: ['versicolor', 'virginica']


2. 請用本章的資料（即用兩個特徵值和 50 筆之後的資料）來做實驗。
我們希望讓預測為 2 的 Virginica 能有最大召回率，因此將機率判斷門檻設為
0，請問：
(1) Virginica 的召回率和精確率，混亂矩陣和綜合報表。請問你觀察到什麼現
象？
(2) 若將 Virginica 的機率判斷門檻設為 0.99，請問其召回率和精確率，混亂矩 陣和綜合報表。請問你觀察到什麼現象？

In [31]:
df = pd.DataFrame(iris['data'], columns=iris['feature_names'])
df['target'] = iris['target']
df = df[['sepal width (cm)', 'petal length (cm)','target']]
df = df.iloc[50:]

X_cols = ['sepal width (cm)', 'petal length (cm)']
y_col = 'target'
X_subset = df[X_cols]
y_subset = df[y_col]

# 分割訓練集和測試集
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_subset, y_subset, test_size=0.33, random_state=42)

#23 用標準化的資料重新檢視正確率
from sklearn.pipeline import make_pipeline
model_pl = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear'))

# 訓練羅吉斯迴歸模型
model_sub = LogisticRegression(solver='liblinear')
model_pl.fit(X_train_sub, y_train_sub)

# 預測測試集
y_pred_proba_sub = model_pl.predict_proba(X_test_sub)[:,1]
y_pred_sub = np.where(y_pred_proba_sub >= 0, 2, 1)

print("Virginica 的召回率和精確率 (門檻為 0):")
print("正確率:", accuracy_score(y_test_sub, y_pred_sub))
print("混淆矩陣:\n", pd.DataFrame(confusion_matrix(y_test_sub, y_pred_sub),
      index=['實際1', '實際2'], columns=['預測1', '預測2']))
print("綜合報告:\n", classification_report(y_test_sub, y_pred_sub))

Virginica 的召回率和精確率 (門檻為 0):
正確率: 0.42424242424242425
混淆矩陣:
      預測1  預測2
實際1    0   19
實際2    0   14
綜合報告:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00        19
           2       0.42      1.00      0.60        14

    accuracy                           0.42        33
   macro avg       0.21      0.50      0.30        33
weighted avg       0.18      0.42      0.25        33



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


當將 Virginica 的機率判定門檻設為 0 時：
對於 Virginica 類别，召回率将是 1，因為將所有樣本都預測為 Virginica，所以所有的 Virginica 樣本都被正確預測。而精確率會受到其他類別樣本的影響，因為所有樣本都預測為 Virginica，而不是所有樣本都是 Virginica，所以精確率不會是最佳的。

In [32]:
# (2) 若將 Virginica 的機率判斷門檻設為 0.99，請問其召回率和精確率，混亂矩 陣和綜合報表。請問你觀察到什麼現象？
y_pred_sub2 = np.where(y_pred_proba_sub>= 0.99, 2, 1)

print("Virginica 的召回率和精確率 (門檻為 0.99):")
print("正確率:", accuracy_score(y_test_sub, y_pred_sub2))
print("混淆矩陣:\n", pd.DataFrame(confusion_matrix(y_test_sub, y_pred_sub2),
      index=['實際1', '實際2'], columns=['預測1', '預測2']))
print("綜合報告:\n", classification_report(y_test_sub, y_pred_sub2))

Virginica 的召回率和精確率 (門檻為 0.99):
正確率: 0.696969696969697
混淆矩陣:
      預測1  預測2
實際1   19    0
實際2   10    4
綜合報告:
               precision    recall  f1-score   support

           1       0.66      1.00      0.79        19
           2       1.00      0.29      0.44        14

    accuracy                           0.70        33
   macro avg       0.83      0.64      0.62        33
weighted avg       0.80      0.70      0.64        33



當將 Virginica 的機率判定門檻設為 0.99 時：
Virginica的召回率降低,但精確率提高。這意味著只有部分Virginica類別的樣本被正確預測為Virginica,但這些預測都是正確的。
